Sources https://towardsdatascience.com/metrics-and-python-850b60710e0c

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = "../input/dont-overfit-ii/train.csv"

In [ ]:
data = pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
data.describe()

# As above we can see most of the series data is in right skewed



# to find the correlation among variables (Multicollinearity)




In [ ]:
data.count()

In [ ]:
data.corr()
cor = data.iloc[:,0:5].corr()
print(cor)


# >  LINEAR REGRESSION

In [ ]:
train, test = train_test_split(data, test_size = 0.3)
print(train.shape)
print(test.shape)


# split the train and test into X and Y variables
# ------------------------------------------------


In [ ]:
train_x = train.iloc[:,0:120]; train_y = train.iloc[:,120]
test_x  = test.iloc[:,0:120];  test_y = test.iloc[:,120]

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


In [ ]:
train_x.head()
train_y.head()


In [ ]:
train_y.head(20)

In [ ]:
train.head()

# ensure that the X variables are all numeric for regression
# ---------------------------------------------

In [ ]:
train.dtypes

# To add the constant term A (Y = A + B1X1 + B2X2 + ... + BnXn)
# Xn = ccomp,slag,flyash.....
# ----------------------------------------------------------


In [ ]:
lm1 = sm.OLS(train_y, train_x).fit()



# Prediction
# -----------------

# store the actual and predicted values in a dataframe for comparison
# --------------------------------------------

In [ ]:
pdct1 = lm1.predict(test_x)
print(pdct1)

In [ ]:
actual = list(test_y.head(50))
type(actual)

In [ ]:
predicted = np.round(np.array(list(pdct1.head(50))),2)
print(predicted)
type(predicted)

# Actual vs Predicted
#-----------------------

In [ ]:
df_results = pd.DataFrame({'actual':actual, 'predicted':predicted})
print(df_results.head(115))

# ##To Check the Accuracy:
#-----------------------------

In [ ]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, pdct1))  
print('Mean Squared Error:', metrics.mean_squared_error(test_y, pdct1))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, pdct1))) 

# MAE : MEAN ABSOLUTE ERROR

# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction. It’s the average over the test sample of the absolute differences between prediction and actual observation where all individual differences have equal weight.

# The mean absolute error uses the same scale as the data. This is known as a scale-dependent accuracy measure and, therefore, cannot be used to make comparisons between series using different scales.



# MSE: MEAN SQUARE ERROR

# MSE is a risk function, corresponding to the expected value of the squared error loss. The fact that MSE is almost always strictly positive (and not zero) is because of randomness or because the estimator does not account for information that could produce a more accurate estimate. The MSE is a measure of the quality of an estimator — it is always non-negative, and values closer to zero are better.

# “Minimizing MSE is a key criterion in selecting estimators: see minimum mean-square error. Among unbiased estimators, minimizing the MSE is equivalent to minimizing the variance, and the estimator that does this is the minimum variance unbiased estimator. However, a biased estimator may have lower MSE; see estimator bias.


# In statistical modelling the MSE can represent the difference between the actual observations and the observation values predicted by the model. In this context, it is used to determine the extent to which the model fits the data as well as whether removing some explanatory variables is possible without significantly harming the model’s predictive ability.”


# RMSE: Root mean square error
 
# RMSE is a quadratic scoring rule that also measures the average magnitude of the error. It’s the square root of the average of squared differences between prediction and actual observation.


# Comparision Root mean square error vs Mean square error

# Similarities:

# Express average model prediction error in same units of the variable of interest.

# Can range from 0 to ∞ and are indifferent to the direction of errors.

# Lower values are better.

# RMSE VS MSE:

# Difference:

# Taking the square root before they are averaged, RMSE gives a relatively high weight to large errors, so RMSE should be useful when large errors are undesirable.

In [ ]:
def RMSE(predict, target):
    return np.sqrt(((predict - target) ** 2).mean())
print ('My RMSE: ' + str(RMSE(test_y,pdct1)) )

# MAPE: Mean absolute percentage error

# Measure of prediction accuracy of a forecasting method in statistics, for example in trend estimation, also used as a loss function for regression problems in machine learning. It usually expresses accuracy as a percentage.

# Although the concept of MAPE sounds very simple and convincing, it has major drawbacks in practical application, and there are many studies on shortcomings and misleading results from MAPE

In [ ]:
def MAPE(predict,target):
    return ( abs((target - predict) / target).mean()) * 100
print ('My MAPE: ' + str(MAPE(test_y,pdct1)) )

# here we go ahead with 60:40 train test split and that mean absoulate percentage error is more than 51% that is inaccurate forcasting.An inaccurate forecast of flurries might not cause a problem if they don't materialize, but an inaccurate forecast of a foot of snow – in either direction – has serious consequences. Inaccurate sales forecasts are legendary. ... They lack a functional, sales-specific CRM or Pipeline Management application.

# RMSLE: Root Mean Squared Logarithmic Error

# In case of RMSLE, you take the log of the predictions and actual values. So basically, what changes is the variance that you are measuring. I believe RMSLE is usually used when you don’t want to penalize huge differences in the predicted and the actual values when both predicted and true values are huge numbers.

# RMSLE measures the ratio between actual and predicted.

# log(pi+1)−log(ai+1)log(pi+1)−log(ai+1)

# can be written as log((pi+1)/(ai+1))log((pi+1)/(ai+1))

# It can be used when you don’t want to penalize huge differences when the values are huge numbers.

# Also, this can be used when you want to penalize under estimates more than over estimates.

In [ ]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(pdct1, test_y):
    assert len(pdct1) == len(test_y)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(test_y)]
    return (sum(terms_to_sum) * (1.0/len(pdct1))) ** 0.5
print ('My RMSLE: ' + str(RMSE(test_y,pdct1)) )

# R² and R-Squared: Coefficient of determination

# R² and R-Squared help us to know how good our regression model as compared to a very simple model that just predicts the mean value of target from the train set as predictions.

In [ ]:
from sklearn.metrics import mean_absolute_error
print ('Sk MAE: ' + str(mean_absolute_error(pdct1,test_y)) )
def MAE(predict,target):
    return (abs(predict-target)).mean()
print ('My MAE: ' + str(MAE(test_y,pdct1)))

In [ ]:
def R2(predict, target):
    return 1 - (MAE(predict,target) / MAE(target.mean(),target))
def R_SQR(predict, target):
    r2 = R2(predict,target)
    return np.sqrt(r2)
print ('My R2         : ' + str(R2(test_y,pdct1)) )
print ('My R          : ' + str(R_SQR(test_y,pdct1)) )

# Adjusted R²

# A model performing equal to baseline would give R-Squared as 0. Better the model, higher the r2 value. The best model with all correct predictions would give R-Squared as 1. However, on adding new features to the model, the R-Squared value either increases or remains the same. R-Squared does not penalize for adding features that add no value to the model. So an improved version over the R-Squared is the adjusted R-Squared

In [ ]:
def R2_ADJ(predict, target, k):
    r2 = R2(predict,target)
    n = len(target)
    return (1 -  ( (1-r2) *  ( (n-1) / (n-(k+1)) ) ) )
k= len(data.columns)
print ('My R2 adjusted: ' + str(R2_ADJ(test_y,pdct1,k)) )